# Imports

In [58]:
import pandas as pd # import for dataframe handle
import numpy as np # import for math and array operations
import matplotlib.pyplot as plt # import for visual representation
#import seaborn as sns # import for visual representation

from bs4 import BeautifulSoup
import requests

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
import string

from sklearn import set_config; set_config(display='diagram')
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from sklearn.compose import make_column_transformer, make_column_selector

#from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import RobustScaler, StandardScaler

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
from nltk.stem import WordNetLemmatizer

def clean_text(text):
    
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') # Remove Punctuation
        
    lowercased = text.lower() # Lower Case
    lemmatizer = WordNetLemmatizer()
    #lemmatized = [lemmatizer.lemmatize(word) for word in lowercased]
    #lowercased = lemmatized     
    tokenized = word_tokenize(lowercased) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('English')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words]# Remove Stop Words
    lemmatized = [lemmatizer.lemmatize(word) for word in without_stopwords]
    
    return " ".join(lemmatized)

# Load Dataset 

In [35]:
df = pd.read_csv('clean_df.csv' )
df.head()

,url,name,developer,tags,languages,genre,game_description,mature_content,price,reviews,date,op_sys,achievements
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,id Software,"FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","English,French,Italian,German,Spanish - Spain,...",Action,"About This Game Developed by id software, the...",0,17.991,Very Positive,2016-05-12 00:00:00,windows,1.0
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,PUBG Corporation,"Survival,Shooter,Multiplayer,Battle Royale,PvP...","English,Korean,Simplified Chinese,French,Germa...","Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,1,26.991,Mixed,2017-12-21 00:00:00,windows,1.0
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,Harebrained Schemes,"Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","English,French,German,Russian","Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,0,35.991,Mostly Positive,2018-04-24 00:00:00,windows macOS Linux,1.0
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,Bohemia Interactive,"Survival,Zombies,Open World,Multiplayer,PvP,Ma...","English,French,Italian,German,Spanish - Spain,...","Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,0,40.491,Mixed,2018-12-13 00:00:00,windows,0.0
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,CCP,"Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","English,German,Russian,French","Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,0,NaN,Mostly Positive,2003-05-06 00:00:00,windows macOS,0.0


In [18]:
df.isnull().sum()

url                     0
name                    3
developer              34
tags                    3
languages               0
genre                   5
game_description        0
mature_content          0
price                4558
reviews                 0
date                  259
op_sys              12383
achievements            0
dtype: int64

In [36]:
df['tags'].fillna('', inplace=True)
df['genre'].fillna('', inplace=True)

In [38]:
df['metadata'] = df[['tags', 'genre']].apply(lambda x: ' '.join(x), axis = 1)


df['metadata'] = df['metadata'].apply(
    lambda x: ' '.join(list(set(x.split())))
)

In [39]:
df.head()

,url,name,developer,tags,languages,genre,game_description,mature_content,price,reviews,date,op_sys,achievements,metadata
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,id Software,"FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","English,French,Italian,German,Spanish - Spain,...",Action,"About This Game Developed by id software, the...",0,17.991,Very Positive,2016-05-12 00:00:00,windows,1.0,"Soundtrack,Multiplayer,Singleplayer,Fast-Paced..."
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,PUBG Corporation,"Survival,Shooter,Multiplayer,Battle Royale,PvP...","English,Korean,Simplified Chinese,French,Germa...","Action,Adventure,Massively Multiplayer",About This Game PLAYERUNKNOWN'S BATTLEGROUND...,1,26.991,Mixed,2017-12-21 00:00:00,windows,1.0,"Co-Op,Tactical,Co-op,First-Person,Early Person..."
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,Harebrained Schemes,"Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","English,French,German,Russian","Action,Adventure,Strategy",About This Game From original BATTLETECH/Mec...,0,35.991,Mostly Positive,2018-04-24 00:00:00,windows macOS Linux,1.0,"Tactics,Sci-fi,Turn-Based Soundtrack,Difficult..."
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,Bohemia Interactive,"Survival,Zombies,Open World,Multiplayer,PvP,Ma...","English,French,Italian,German,Spanish - Spain,...","Action,Adventure,Massively Multiplayer",About This Game The post-soviet country of Ch...,0,40.491,Mixed,2018-12-13 00:00:00,windows,0.0,"Access,Simulation,FPS,Post-apocalyptic,Surviva..."
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,CCP,"Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","English,German,Russian,French","Action,Free to Play,Massively Multiplayer,RPG,...",About This Game,0,NaN,Mostly Positive,2003-05-06 00:00:00,windows macOS,0.0,"to Multiplayer,Sci-fi,Sandbox,MMORPG,Open Sim,..."


In [52]:
df['metadata'] = df['metadata'].apply(clean_text)

In [53]:
df['game_description'] = df['game_description'].apply(clean_text)

In [54]:
df.head()

,url,name,developer,tags,languages,genre,game_description,mature_content,price,reviews,date,op_sys,achievements,metadata
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,id Software,"FPS,Gore,Action,Demons,Shooter,First-Person,Gr...","English,French,Italian,German,Spanish - Spain,...",Action,game developed id software studio pioneered fi...,0,17.991,Very Positive,2016-05-12 00:00:00,windows,1.0,soundtrack multiplayer singleplayer fast paced...
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,PUBG Corporation,"Survival,Shooter,Multiplayer,Battle Royale,PvP...","English,Korean,Simplified Chinese,French,Germa...","Action,Adventure,Massively Multiplayer",game playerunknown battleground battle royale ...,1,26.991,Mixed,2017-12-21 00:00:00,windows,1.0,co op tactical co op first person early person...
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,Harebrained Schemes,"Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","English,French,German,Russian","Action,Adventure,Strategy",game original battletech mechwarrior creator j...,0,35.991,Mostly Positive,2018-04-24 00:00:00,windows macOS Linux,1.0,tactic sci fi turn based soundtrack difficult ...
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,Bohemia Interactive,"Survival,Zombies,Open World,Multiplayer,PvP,Ma...","English,French,Italian,German,Spanish - Spain,...","Action,Adventure,Massively Multiplayer",game post soviet country chernarus struck unkn...,0,40.491,Mixed,2018-12-13 00:00:00,windows,0.0,access simulation fps post apocalyptic surviva...
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,CCP,"Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","English,German,Russian,French","Action,Free to Play,Massively Multiplayer,RPG,...",game,0,NaN,Mostly Positive,2003-05-06 00:00:00,windows macOS,0.0,multiplayer sci fi sandbox mmorpg open sim sim...


# Preprocessing

In [57]:
reviews_rating = [
    "Overwhelmingly Negative",
    "Very Negative",
    "Negative",
    "Mostly Negative",
    'Mixed',
    "Mostly Positive",
    "Positive",
    "Very Positive",
    "Overwhelmingly Positive"
]

In [76]:
text_transformer_1 = make_pipeline(TfidfVectorizer(min_df=0.1), RobustScaler())
text_col_meta = make_column_selector('metadata')


text_transformer_2 = make_pipeline(TfidfVectorizer(min_df=0.05), RobustScaler())
text_col_desc = make_column_selector('game_description')


ord_transformer = make_pipeline(
    OrdinalEncoder(
        categories=reviews_rating,
        dtype=np.int64,
        handle_unknown="use_encoded_valeu",
        unknown_value=-1
    ), StandardScaler())
ord_col = make_column_selector('reviews')


num_transformer = make_pipeline(StandardScaler())
num_col = make_column_selector(['mature_content', 'achievements'])


preproc_basic = make_column_transformer(
    (text_transformer_1, text_col_meta),
    (text_transformer_2, text_col_desc),
    (ord_transformer, ord_col),
    (num_transformer, num_col),
    remainder='drop'
)

preproc_basic

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('tfidfvectorizer',
                                                  TfidfVectorizer(min_df=0.1)),
                                                 ('robustscaler',
                                                  RobustScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x1662b2760>),
                                ('pipeline-2',
                                 Pipeline(steps=[('tfidfvectorizer',
                                                  TfidfVectorizer(min_df=0.05)),
                                                 ('robustscaler',
                                                  RobustScaler())]),
                                 <sklearn.compo...
                                                                             'Positive'],
                                                                 dtype=<class 'numpy.int64'>,
                                                                 handle_unknown='use_encoded_valeu',
                                                                 unknown_value=-1)),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x1662b3a00>),
                                ('pipeline-4',
                                 Pipeline(steps=[('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x1662b9280>)])

In [77]:
preproc_basic.fit_transform(df)

TypeError: unhashable type: 'list'